In [ ]:
# !pip install matplotlib numpy

In [ ]:
import os
from typing import List, Tuple
from matplotlib import pyplot as plt
import numpy as np

files_to_read = ['rqc_off_stats.txt', 'rqc_on_stats copy.txt']

def read_file(file_name):
    with open(file_name) as f:
        lines = f.readlines()
        return lines

def get_rocksdb_compact_write_bytes(lines) -> List[str]:
    return [line for line in lines if 'rocksdb.compact.write.bytes' in line]

def get_rocksdb_flush_write_bytes(lines) -> List[str]:
    return [line for line in lines if 'rocksdb.flush.write.bytes' in line]

def transform_bytes(lines) -> List[int]:
    return [int(line.strip(' \n').split(' : ')[1]) for line in lines]

def get_total_bytes(compact_bytes, flush_bytes) -> List[int]:
    return [compact_bytes[i] + flush_bytes[i] for i in range(len(compact_bytes))]

def get_levels_stats(lines) -> List[Tuple[str, int, int]]:
    return ["Level       No. of Files       No. of Entries\n"] + [line.strip(' \t') for line in lines if 'Level-' in line or 'Total:' in line]


In [ ]:
experiments = os.listdir('./')

results = list()

for experiment in experiments:

    if not os.path.isdir(experiment):
        continue
    
    i, inserts, u, updates, s, range_queries, y, selectivity, t, size_ratio, *others = experiment.split(' ')

    if len(others) == 2:
        rq, rqc = others
        lb, lb_value = None, 0
        ub, ub_value = None, 0
    else:
        lb, lb_value, ub, ub_value, rq, rqc = others

    if rqc =='0':
        approach = 'vanilla'
        lines = read_file(f'{experiment}/rqc_off_stats.txt')
    else:
        approach = 'rqdc'
        lines = read_file(f'{experiment}/rqc_on_stats.txt')
    
    compact_write_bytes = get_rocksdb_compact_write_bytes(lines)
    flush_write_bytes = get_rocksdb_flush_write_bytes(lines)
    levels_details = get_levels_stats(lines)

    compact_bytes = transform_bytes(compact_write_bytes)
    flush_bytes = transform_bytes(flush_write_bytes)

    results.append({
        'inserts': inserts,
        'updates': updates,
        'range_queries': range_queries,
        'size_ratio': size_ratio,
        'rqc': rqc,
        'lb': lb_value,
        'ub': ub_value,
        'approach': approach,
        'compact_write_bytes': compact_bytes,
        'flush_write_bytes': flush_bytes,
        'total_write_bytes': get_total_bytes(compact_bytes, flush_bytes),
        'levels_details': levels_details,
    })

In [ ]:
lb_ub_values_rqdc = list()
for result in results:
    if result['lb'] == '0' and result['ub'] == '0':
        continue
    lb_ub_values_rqdc.append((result['lb'], result['ub']))

In [ ]:
size_ratios = ['4']
utl_ltu_value_vanilla = [(0, 0)]

for size_ratio in size_ratios:
    rqdc_data = [exp for exp in results if exp['approach'] == 'rqdc' and exp['size_ratio'] == size_ratio]
    vanilla_data = [exp for exp in results if exp['approach'] == 'vanilla' and exp['size_ratio'] == size_ratio]

    for lb, ub in lb_ub_values_rqdc:
        rqdc_data_lb_ub = [exp for exp in rqdc_data if exp['lb'] == str(lb) and exp['ub'] == str(ub)]
        rqdc_epochs = range(1, len(rqdc_data_lb_ub[0]['total_write_bytes']) + 1)
        rqdc_total_bytes = rqdc_data_lb_ub[0]['total_write_bytes']
        vanilla_total_bytes = vanilla_data[0]['total_write_bytes']

        vanilla_levels_info_string = "                 Vanilla\n" + "".join(vanilla_data[0]['levels_details'])
        info_str = vanilla_levels_info_string + "\n\n                   RQDC\n" + "".join(rqdc_data_lb_ub[0]['levels_details'])

        plt.figure(figsize=(10, 6))
        plt.plot(rqdc_epochs, rqdc_total_bytes, label=f'rqdc - lb: {lb} ub: {ub}', marker='o', color='blue')
        plt.plot(rqdc_epochs, vanilla_total_bytes, label='vanilla', marker='s', color='orange')
        plt.title(f'Size Ratio {size_ratio}, Lower Bound {lb}, Upper Bound {ub}')
        plt.xlabel('Epoch')
        plt.ylabel('Total Bytes Written')
        plt.legend()
        plt.grid(True)

        plt.text(1.02, 0.5, info_str, transform=plt.gca().transAxes, fontsize=10, verticalalignment='center', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
        plt.show()